In [ ]:
lineups = {
	'Giovanni': {
		'poke1': ['Shadow Persian'],
		'poke2': ['Shadow Kangaskhan', 'Shadow Rhyperior', 'Shadow Nidoking'],
		'poke3': ['Shadow Dialga'],
	},
	'Arlo': {
		'poke1': ['Shadow Horsea'],
		'poke2': ['Shadow Slowbro', 'Shadow Crobat', 'Shadow Steelix'],
		'poke3': ['Shadow Scizor', 'Shadow Alakazam', 'Shadow Alolan Muk'],
	},
	# 'Sierra': {
	# 	'poke1': ['Shadow '],
	# 	'poke2': ['Shadow ', 'Shadow ', 'Shadow '],
	# 	'poke3': ['Shadow ', 'Shadow ', 'Shadow '],
	# },
	# 'Cliff': {
	# 	'poke1': ['Shadow '],
	# 	'poke2': ['Shadow ', 'Shadow ', 'Shadow '],
	# 	'poke3': ['Shadow ', 'Shadow ', 'Shadow '],
	# },
	"Normal doesn't mean weak.": {
		'poke1': ['Shadow Bunnelby', 'Shadow Bidoof', 'Shadow Slakoth'],
		'poke2': ['Shadow Diggersby', 'Shadow Rattata', 'Shadow Vigoroth'],
		'poke3': ['Shadow Ursaring', 'Shadow Raticate', 'Shadow Slaking'],
	},
	# '': {
	# 	'poke1': ['Shadow ', 'Shadow ', 'Shadow '],
	# 	'poke2': ['Shadow ', 'Shadow ', 'Shadow '],
	# 	'poke3': ['Shadow ', 'Shadow ', 'Shadow '],
	# },
	# '': {
	# 	'poke1': ['Shadow ', 'Shadow ', 'Shadow '],
	# 	'poke2': ['Shadow ', 'Shadow ', 'Shadow '],
	# 	'poke3': ['Shadow ', 'Shadow ', 'Shadow '],
	# },
	# '': {
	# 	'poke1': ['Shadow ', 'Shadow ', 'Shadow '],
	# 	'poke2': ['Shadow ', 'Shadow ', 'Shadow '],
	# 	'poke3': ['Shadow ', 'Shadow ', 'Shadow '],
	# }
}

In [ ]:
import requests

from bs4 import BeautifulSoup
from urllib.parse import urlencode

headers = { 'User-Agent': 'Mozilla/5.0' }
gamepress_root = 'https://pokemongo.gamepress.gg'

def get_pokemons():
	params = { '_data': '_custom/routes/_site.c.pokemon+/_index' }
	res = requests.get(f'{gamepress_root}/c/pokemon?{urlencode(params)}', headers=headers)
	return res.json().get('list', {}).get('listData', {}).get('docs')

def get_pokemon_type_chart(gamepass_pokemon_id):
	res = requests.get('https://pokemongo.gamepress.gg/c/pokemon/shadow-dialga')
	soup = BeautifulSoup(res.text, 'html.parser')
	tables = soup.select("table")
	type_chart = {}
	for table in tables:
		title = table.select('thead>tr>th>button>svg>title')
		if len(title) != 1:
			continue
		type_chart[title[0].text.lower()] = [_.select_one('td>a>div>span').text for _ in table.select('tbody>tr')]
	return type_chart

def get_lineups_data(lineups):
	pokes_data = {}
	for team_go, pokemons in lineups.items():
		all_pokes = pokemons.get('poke1') + pokemons.get('poke2') + pokemons.get('poke3')

		for all_poke in all_pokes:
			pokemon_id = all_poke.lower().replace(' ', '-')
			pokes_data[all_poke] = get_pokemon_type_chart(pokemon_id)
	return pokes_data

pokemons = get_pokemons()
print(f'Found {len(pokemons)} pokemons')

In [ ]:
lineups_data = get_lineups_data(lineups)
print(f'Found {len(lineups_data.keys())} lineups data')

In [ ]:
for trainer, pokemons in lineups.items():
	trainer_pokemons = {}
	vulnerables = {}

	poke1 = pokemons.get('poke1')
	poke2 = pokemons.get('poke2')
	poke3 = pokemons.get('poke3')

	for poke in poke1 + poke2 + poke3:
		poke_vulnerable = lineups_data[poke].get('vulnerable')
		trainer_pokemons[poke] = { 'vulnerable': poke_vulnerable }
		# vulnerables += poke_vulnerable
		for poke_vul in poke_vulnerable:
			counter = vulnerables.get(poke_vul, 0)
			vulnerables[poke_vul] = counter + 1
	print(trainer, vulnerables)